In [30]:
import pandas as pd
import pickle
import subprocess
import os

In [2]:
data = pd.read_csv("../H3K27me3/GSM2188866_MDA231_merged.ghits.pkbin.net.txt", sep="\t", header=None)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8
0,chr1,1785285,1891117,ENSG00000078369,22490,-,chr1,908000,55.2273
1,chr1,1785285,1891117,ENSG00000078369,22490,-,chr1,909000,55.2273
2,chr1,1785285,1891117,ENSG00000078369,22490,-,chr1,910000,41.3955
3,chr1,1785285,1891117,ENSG00000078369,22490,-,chr1,911000,55.2273
4,chr1,1785285,1891117,ENSG00000078369,22490,-,chr1,912000,55.2273


In [3]:
#Наши РНК
with open("../H3K27me3/lncRNA_All_Peaks_Correlations_corrected.pickle", 'rb') as f:
    corr = pickle.load(f)
corr = [i for i in corr if len(i) != 0]

In [4]:
len(corr)

1337

In [5]:
peaks = pd.read_csv("../H3K27me3/peaks/merged_peaks.bed", sep="\t", header=None)

In [6]:
lncRNAs = [c[0][0].split('.')[0] for c in corr]

In [7]:
commonRNAs = set(lncRNAs).intersection(set(data[3].tolist()))

In [8]:
len(commonRNAs)

35

In [4]:
#Сделать bed файлы с участками ДНК по РНК

In [9]:
for rna in commonRNAs:
    df = data[data[3] == rna][[6, 7]]
    df[8] = data[7] + 1000
    df.to_csv("../H3K27me3/grid_seq/" + rna + ".bed", sep="\t", header=None, index=None)

In [10]:
peaks.head()

,0,1,2,3
0,chr1,10341,10640,peak_0
1,chr1,54315,55724,peak_1
2,chr1,56893,57208,peak_2
3,chr1,60654,60893,peak_3
4,chr1,61834,62068,peak_4


In [ ]:
#Сделать bed файлы с пиками по РНК

In [18]:
from collections import defaultdict
d = defaultdict(list)
for i in [i for c in corr for i in c if i[0].split('.')[0] in commonRNAs]:
    d[i[0].split('.')[0]].append(i[1])

In [19]:
for l in commonRNAs:
    peaks_names = d[l]
    lnc_peaks = peaks[peaks[3].isin(peaks_names)]
    lnc_peaks.to_csv("../H3K27me3/all_peaks_coords_by_lnc/" + l + ".bed", sep="\t", header=None, index=None)

In [21]:
data = pd.read_csv("../H3K27me3/all_peaks_coords_by_lnc/liftToHg38_ENSG00000230551.bed", sep="\t", header=None)

In [22]:
data[0].value_counts()

chr12    1
Name: 0, dtype: int64

In [ ]:
#liftOver все наши файлы, потому что данные grid_seq в hg38

In [11]:
def liftOver(bed, liftover_bed, unlifted_bed):
    try:
        output = subprocess.check_output("liftOver " + bed + " ../hg19ToHg38.over.chain.gz " + liftover_bed + " " + unlifted_bed, shell=True, stderr=subprocess.STDOUT).decode()
    except subprocess.CalledProcessError as e:
        output = e.output.decode()        

    print(output)    

In [13]:
files = os.listdir("../H3K27me3/all_peaks_coords_by_lnc")
for l in files:
    liftOver("../H3K27me3/all_peaks_coords_by_lnc/" + l, "../H3K27me3/all_peaks_coords_by_lnc/liftToHg38_" + l, "../H3K27me3/all_peaks_coords_by_lnc/unliftToHg38_" + l) 

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading liftover chains
Mapping coordinates

Reading li

In [ ]:
#Объединяем результаты genometricCirr в один файл

In [49]:
frames = []
for f in os.listdir("../H3K27me3/genometric_results/"):
    if f.endswith('tsv'):
        lnc_name = f.split('.')[0].split('_')[1]
        frame = pd.read_csv("../H3K27me3/genometric_results/" + f, sep='\t')
        frame['lnc'] = [lnc_name]*frame.shape[0]
        frames.append(frame)

In [53]:
df = pd.concat(frames)

In [57]:
df.set_index(['lnc', 'chr.names']).to_csv("../H3K27me3/genometric_results/all_rnas.tsv", sep="\t")

In [58]:
pd.read_csv("../H3K27me3/genometric_results/all_rnas.tsv", sep="\t")

,lnc,chr.names,query.population,reference.population,relative.distances.ks.p.value,relative.distances.ecdf.deviation.area,relative.distances.ecdf.area.correlation,relative.distances.ecdf.deviation.area.p.value,scaled.absolute.min.distance.sum.p.value,scaled.absolute.min.distance.sum.lower.tail,jaccard.measure.p.value,jaccard.measure.lower.tail,projection.test.p.value,projection.test.lower.tail,projection.test.obs.to.exp
0,ENSG00000269900,chr1,65,7369,3.162442e-01,0.013221,0.016970,0.77,<0.01,True,<0.01,True,1.023722e-04,True,0.370400
1,ENSG00000269900,chr10,19,3577,1.480467e-03,0.104753,0.420366,<0.01,<0.01,True,0.07,False,3.409949e-02,False,1.507284
2,ENSG00000269900,chr11,62,3958,2.438352e-01,0.025009,-0.099500,0.23,<0.01,True,<0.01,False,1.467376e-04,False,1.602268
3,ENSG00000269900,chr12,99,4311,7.515214e-01,0.011323,0.006171,0.61,0.05,True,<0.01,True,4.882966e-06,True,0.347662
4,ENSG00000269900,chr14,33,2712,7.638953e-07,0.089946,0.329757,<0.01,0.01,True,<0.01,True,5.479367e-03,True,0.311328
5,ENSG00000269900,chr15,6,2456,9.065126e-01,0.033727,0.067297,0.94,<0.01,True,<0.01,False,0.000000e+00,False,3.262576
6,ENSG00000269900,chr17,29,2272,1.126508e-02,0.060745,0.224916,0.01,<0.01,True,<0.01,False,1.816450e-06,False,2.135070
7,ENSG00000269900,chr19,27,2087,2.804161e-08,0.127537,0.509436,<0.01,0.02,True,<0.01,False,2.882058e-09,False,2.766237
8,ENSG00000269900,chr2,46,6905,1.064703e-01,0.041731,0.166326,0.05,0.03,False,0.01,False,1.149204e-02,False,1.481669
9,ENSG00000269900,chr20,85,1267,3.545775e-04,0.057229,0.158448,<0.01,<0.01,True,0.01,False,5.563481e-03,False,1.280577
